# PhenoAge

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.PhenoAge)

class PhenoAge(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies a convertion from a CDF of the mortality score from a Gompertz
        distribution to phenotypic age.
        """
        # lambda
        l = torch.tensor(0.0192, device=x.device, dtype=x.dtype)
        mortality_score = 1 - torch.exp(-torch.exp(x) * (torch.exp(120 * l) - 1) / l)
        age = (
            141.50225 + torch.log(-0.00553 * torch.log(1 - mortality_score)) / 0.090165
        )
        return age



In [3]:
model = pya.models.PhenoAge()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'phenoage'
model.metadata["data_type"] = 'blood chemistry'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2018
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Levine, Morgan E., et al. \"An epigenetic biomarker of aging for lifespan and healthspan.\" Aging (albany NY) 10.4 (2018): 573."
model.metadata["doi"] = "https://doi.org/10.18632%2Faging.101414"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
features = [
    "albumin",
    "creatinine",
    "glucose",
    "log_crp",
    "lymphocyte_percent",
    "mean_cell_volume",
    "red_cell_distribution_width",
    "alkaline_phosphatase",
    "white_blood_cell_count",
    "age"
]

coefs = [
    -0.0336,
    0.0095,
    0.1953,
    0.0954,
    -0.0120,
    0.0268,
    0.3306,
    0.0019,
    0.0554,
    0.0804,
]

## Load features

In [6]:
model.features = features

## Load weights into base model

In [7]:
weights = torch.tensor(coefs).unsqueeze(0)
intercept = torch.tensor([-19.9067])

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = 'mortality_to_phenoage'
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Levine, Morgan E., et al. "An epigenetic biomarker of aging for '
             'lifespan and healthspan." Aging (albany NY) 10.4 (2018): 573.',
 'clock_name': 'phenoage',
 'data_type': 'blood chemistry',
 'doi': 'https://doi.org/10.18632%2Faging.101414',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2018}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: 'mortality_to_phenoage'
postprocess_dependencies: None
features: ['albumin',
 'creatinine',
 'glucose',
 'log_crp',
 'lymphocyte_percent',
 'mean_cell_volume',
 'red_cell_distribution_width',
 'alkaline_phosphatase',
 'white_blood_cell_count',
 'age']
base_model_features: None

%==================================== Model Details ====================================%
Model

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-74.5299],
        [-69.6854],
        [-69.2747],
        [-59.3164],
        [-64.4732],
        [-62.6114],
        [-72.9975],
        [-65.8399],
        [-69.8485],
        [-69.8503]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)